# Factor Model Estimator

Factor Model impose a **structure** on asset's covariance matrix by using factors to explain their relationships.
+ Reduce the dimensionality of estimation.
+ Robust against noise
+ Decompose risk into systematic and idiosyncratic components

## Data

Use 5 ETF's daily prices as common factors.

In [3]:
from plotly.io import show
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split

from skfolio import Population, RiskMeasure
from skfolio.datasets import load_factors_dataset, load_sp500_dataset
from skfolio.moments import GerberCovariance, ShrunkMu
from skfolio.optimization import MeanRisk, ObjectiveFunction
from skfolio.preprocessing import prices_to_returns
from skfolio.prior import EmpiricalPrior, FactorModel, LoadingMatrixRegression

prices = load_sp500_dataset()
factor_prices = load_factors_dataset()

X, y = prices_to_returns(prices, factor_prices)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)

In [6]:
X_train.head()

,AAPL,AMD,BAC,BBY,CVX,GE,HD,JNJ,JPM,KO,LLY,MRK,MSFT,PEP,PFE,PG,RRC,UNH,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,
2014-01-03,-0.021941,0.012658,0.019298,0.004423,0.001694,-0.000724,-0.001591,0.009005,0.007727,-0.004921,0.007290,0.004831,-0.006747,0.001696,0.001976,-0.001119,-0.014801,0.007104,-0.003300,-0.002409
2014-01-06,0.005417,0.032500,0.015233,-0.031223,-0.002663,-0.008007,-0.009639,0.005223,0.005801,-0.004678,0.008424,0.000198,-0.021116,0.000495,0.000986,0.002372,0.006461,-0.011444,-0.005591,0.001503
2014-01-07,-0.007145,0.012107,-0.009646,-0.026135,0.008466,0.001096,0.004920,0.021230,-0.011535,0.002954,-0.006601,0.007464,0.007758,0.014579,0.006206,0.009661,0.017028,0.030569,0.003078,0.014147
2014-01-08,0.006311,0.000000,0.004834,-0.014072,-0.014226,-0.002926,0.005277,-0.001371,0.009434,-0.011147,-0.001571,-0.006399,-0.017865,-0.002880,0.006853,-0.014483,0.001090,-0.011626,-0.007907,-0.003259
2014-01-09,-0.012719,-0.021531,0.015078,-0.008176,0.000000,0.000366,-0.004385,0.006056,-0.001860,-0.005247,0.011157,-0.005423,-0.006449,-0.004688,-0.000681,0.002246,-0.015635,0.006088,0.003346,-0.009735


In [7]:
y_train.head()

,MTUM,QUAL,SIZE,USMV,VLUE
Date,,,,,
2014-01-03,0.001670,-0.001965,-0.005389,-0.000273,-0.001169
2014-01-06,-0.002178,-0.003917,0.000000,-0.002284,-0.000170
2014-01-07,0.008258,0.008072,0.000185,0.005707,0.005576
2014-01-08,0.007343,-0.000371,0.000000,-0.000272,0.000000
2014-01-09,0.001832,-0.000537,-0.002093,0.001428,-0.002180


## Model

In [8]:
max_sharpe_model_factor_1 = MeanRisk(
    risk_measure=RiskMeasure.VARIANCE,
    objective_function=ObjectiveFunction.MAXIMIZE_RATIO,
    prior_estimator=FactorModel(),
    portfolio_params=dict(name="Factor Model 1")
)
max_sharpe_model_factor_1.fit(X_train, y_train)
max_sharpe_model_factor_1.weights_

array([1.03294289e-06, 1.27482685e-03, 4.19682803e-07, 3.34130825e-06,
       7.36838286e-07, 1.28824408e-06, 5.13031432e-02, 6.35619183e-02,
       6.14804833e-07, 1.79106051e-01, 5.03130911e-02, 7.13734379e-02,
       4.13002526e-02, 2.27978407e-01, 5.13348034e-02, 1.44130375e-01,
       2.99026116e-07, 6.19737850e-02, 5.63413085e-02, 8.67773196e-07])

In [9]:
max_sharpe_model_factor_2 = MeanRisk(
    risk_measure=RiskMeasure.VARIANCE,
    objective_function=ObjectiveFunction.MAXIMIZE_RATIO,
    prior_estimator=FactorModel(
        loading_matrix_estimator=LoadingMatrixRegression(
            linear_regressor=RidgeCV(fit_intercept=False), n_jobs=-1
        )
    ),
    portfolio_params=dict(name="Factor Model 2")
)
max_sharpe_model_factor_2.fit(X_train, y_train)
max_sharpe_model_factor_2.weights_

array([3.97758339e-02, 6.57843874e-03, 2.18405141e-02, 8.98258882e-03,
       3.16197378e-02, 1.42391168e-02, 8.00124906e-02, 8.32090802e-02,
       4.74782930e-02, 8.59470407e-02, 4.59776221e-02, 5.91778878e-02,
       8.42236770e-02, 1.05684777e-01, 6.43841778e-02, 7.94729901e-02,
       3.76786711e-05, 5.23695742e-02, 4.35215146e-02, 4.54669667e-02])

Empirical prior estimator of factors.

In [10]:
max_sharpe_model_factor_3 = MeanRisk(
    risk_measure=RiskMeasure.VARIANCE,
    objective_function=ObjectiveFunction.MAXIMIZE_RATIO,
    prior_estimator=FactorModel(
        factor_prior_estimator=EmpiricalPrior(
            mu_estimator=ShrunkMu(), covariance_estimator=GerberCovariance()
        )
    ),
    portfolio_params=dict(name="Factor Model 3")
)
max_sharpe_model_factor_3.fit(X_train, y_train)
max_sharpe_model_factor_3.weights_

array([4.86490688e-07, 4.38230191e-07, 4.24408219e-08, 6.69653310e-08,
       5.11878211e-08, 6.14581598e-08, 1.68436387e-02, 2.08439608e-06,
       5.27854151e-08, 6.45513596e-02, 6.24004728e-02, 9.61498232e-02,
       3.68209826e-01, 2.44692220e-01, 5.86512134e-07, 9.30385158e-06,
       2.19939734e-08, 1.47139096e-01, 3.09340377e-07, 5.81316429e-08])

In [18]:
prior_estimator = max_sharpe_model_factor_3.prior_estimator_
prior_estimator


FactorModel(factor_prior_estimator=EmpiricalPrior(covariance_estimator=GerberCovariance(),
                                                  mu_estimator=ShrunkMu()))

In [19]:
prior_model = prior_estimator.prior_model_
prior_model

PriorModel(mu=array([ 0.00059752,  0.00078836, -0.00053323, -0.00012697, -0.00033297,
       -0.00021685,  0.00061879,  0.00054947, -0.00029846,  0.00050157,
        0.00068264,  0.00069036,  0.00125097,  0.00065547,  0.00045871,
        0.00049221, -0.00175945,  0.00081062,  0.00033362, -0.00023231]), covariance=array([[ 2.75022074e-04,  1.29310397e-04,  1.14094740e-04,
         6.75191637e-05,  7.20639440e-05,  7.33237546e-05,
         6.16476554e-05,  4.68291134e-05,  9.48837814e-05,
        -5.26599038e-06,  3.48607146e-05,  3.01473788e-05,
         1.15046221e-04,  1.40148901e-07,  3.44889698e-05,
         8.52261680e-07,  1.05781045e-04,  2.90934542e-05,
         3.88259728e-06,  7.75978321e-05],
       [ 1.29310397e-04,  1.45566967e-03,  1.55981389e-04,
         1.10954429e-04,  7.64768830e-05,  8.67241392e-05,
         8.08813989e-05,  3.05345368e-05,  1.21105046e-04,
        -4.89395055e-06,  4.82541428e-05,  4.51634980e-05,
         1.27995691e-04, -4.55732351e-06,  5.6655570

In [20]:
loading_matrix = prior_estimator.loading_matrix_estimator_.loading_matrix_
loading_matrix

array([[ 0.34905954,  1.59051502, -0.05077361, -0.99167193, -0.        ],
       [ 1.16592858,  0.        ,  0.28177423, -0.82678691,  0.92261137],
       [ 0.06342149,  0.68820926, -0.        , -0.90387695,  1.17904377],
       [ 0.        ,  0.19084808,  0.08086552, -0.        ,  0.82730174],
       [-0.51036765,  0.78697298, -0.16877212,  0.25234305,  0.6829133 ],
       [-0.28101136,  0.64151033,  0.03860762, -0.        ,  0.59636878],
       [ 0.17691041,  0.36515599,  0.00321386,  0.32634051,  0.15390387],
       [-0.278366  ,  0.70692514, -0.17666442,  0.78353708, -0.12399197],
       [-0.10659571,  0.69754112, -0.06647323, -0.35735584,  0.8785099 ],
       [-0.41056254, -0.00260142, -0.01212348,  1.46081088, -0.12137341],
       [ 0.        ,  0.18620599, -0.        ,  0.82933288, -0.        ],
       [-0.        ,  0.11966633,  0.        ,  0.88930418,  0.        ],
       [ 0.69202998,  0.89393433, -0.15846805, -0.22400802, -0.07128371],
       [-0.3957795 ,  0.10853663, -0.0

In [21]:
benchmark_model = MeanRisk(
    risk_measure=RiskMeasure.VARIANCE,
    objective_function=ObjectiveFunction.MAXIMIZE_RATIO,
    portfolio_params=dict(name="Empirical")
)
benchmark_model.fit(X_train)
benchmark_model.weights_

array([1.01561518e-01, 7.81165193e-02, 6.29030036e-07, 1.89005488e-02,
       3.05610119e-07, 1.55770502e-07, 1.10594710e-01, 1.22328443e-06,
       1.56471742e-06, 3.39453276e-06, 1.62631058e-01, 1.92171374e-06,
       1.77783711e-01, 9.61805760e-02, 4.64493062e-07, 9.68566446e-03,
       7.41771352e-08, 2.44533886e-01, 1.83984287e-06, 2.34178301e-07])

## Prediction

In [22]:
pred_factor_1 = max_sharpe_model_factor_1.predict(X_test)
pred_factor_2 = max_sharpe_model_factor_2.predict(X_test)
pred_factor_3 = max_sharpe_model_factor_3.predict(X_test)
pred_benchmark = benchmark_model.predict(X_test)

## Analysis

In [24]:
population = Population(
    [
        pred_factor_1,
        pred_factor_2,
        pred_factor_3,
        pred_benchmark
    ]
)
population.plot_cumulative_returns()

In [25]:
population.plot_composition()